# 11 A Pythonic Object
Some notes, observations and questions along chapter 11.

### Object Representations
- `repr()` - Return a string representing the object as the developer wants to see it. It’s what you get when the Python console or a debugger shows an object.

- `str()` - Return a string representing the object as the user wants to see it. It’s what you get when you print() an object.

### Vector Class Redux
This is an example class that we will use in the next chapters to investigate special methods:

In [2]:
from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +       # To generate bytes, we convert the typecode to bytes and concatenate…
                bytes(array(self.typecode, self)))  # …bytes converted from an array built by iterating over the instance.

    def __eq__(self, other):
        return tuple(self) == tuple(other)          # to quickly compare all components, build tuples out of the operands

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

It should work like this:

In [3]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [7]:
v1

Vector2d(3.0, 4.0)

In [9]:
print(v1)

(3.0, 4.0)


In [4]:
# __iter__ makes unpacking work:
x, y = v1
x, y

(3.0, 4.0)

In [8]:
octets = bytes(v1)
octets
# not really sure why we build it like this / why it needs to look like this

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [12]:
# Using eval here shows that the repr of a Vector2d is a faithful representation of its constructor call:
v1_clone = eval(repr(v1))
v1 == v1_clone # possible because we have implemented `__eq__`

True

In [10]:
abs(v1)

5.0

In [11]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

### An Alternative Constructor
- we can export Vector2d as bytes, so here we want to re-construct the object from the bytes
- we could do this in a `classmethod` `frombytes()`:

In [13]:
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

### classmethod versus staticmethod
- `classmethod` changes the way the method is called, so it receives the class itself as the first argument, instead of an instance
    - most commonly used for alternative constructors
- `staticmethod` decorator changes a method so that it receives no special first argument
    - it defines a function on the level/scope of the module and is only informally connected to the class

In [26]:
class Demo:
    @classmethod
    def klassmeth(*args): # *args packs all the positional arguments, also `cls`
        # returns all the positional arguments
        return args
    
    @staticmethod
    def statmeth(*args):
        # returns all the positional arguments
        return args
    
    def f(): # this would also work without the `self`, because it is there by default
        return 1

Demo.klassmeth()

(__main__.Demo,)

In [15]:
# receives the Demo class as the first argument, no matter how we invoke it
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [16]:
Demo.statmeth()

()

In [17]:
# behaves just like a plain old function
Demo.statmeth('spam')

('spam',)

### Formatted Displays
- uses Format Specification Mini-Language (which takes a while to get used to and is not my focus right now)
- implementing our own format method in the `Vector2d` class:

In [18]:
    def __format__(self, fmt_spec=''):
        # uses the formatting specification on each float component in our vector
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components) # plug the formatted strings into the format (x,y)

Now we want to add a custom formatting code, using `p` because it is used anywhere else (and hence cannot be confused).

We want to display the vector in polar coordinates: `<r, θ>`, where `r` is the magnitude and `θ` (theta) is the angle in radians:

In [20]:
# calculating the angle
    def angle(self):
            return math.atan2(self.y, self.x)

# enhancing the __format__ method to take the formatting code into account:
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle()) # we use `self.__abs__()` and `self.angle()`
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})' # preparing the string for returning
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

IndentationError: unexpected indent (403098287.py, line 2)

### A Hashable Vector2d
- make our `Vector2d` hashable, so we can build sets of vectors, or use them as dict keys

In [22]:
# so far, our vector objects are not hashable:
v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

- to make them hashable, we must implement `__hash__` and `__eq__` (which we already did) and make them immutable, so that `v1.x = 7` would raise

In [2]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property # The @property decorator marks the getter method of a property.
    def x(self): # The getter method is named after the public property it exposes: x.
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

This is an example for read-only properties. We would internally rename the `x` and `y` into `self.__x` and `self.__y` and allow access only through functions `x()` and `y()` that only allow getting, but not setting.

In [3]:
# we can only get, but not set x and y
v = Vector2d(2,3)
v.x=1

AttributeError: property 'x' of 'Vector2d' object has no setter

Our Vector2d is now reasonably immutable (which in theory is not strictly necessary, but recommended), so we can define a hash. It is recommended to calculate the hash from the object's components. (Also, ideally we need to take care of `__eq__` to make sure that objects evaluating to equal have the same hashes.)

In [4]:
    def __hash__(self):
        return hash((self.x, self.y))

In [5]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(8587317881251, 8587317881248)

### Supporting Positional Pattern Matching
- `__match_args__` class attribute is used to use positional args in pattern matching

### Complete Listing of Vector2d
- summary of what we have done with Vector2d

In [10]:
from array import array
import math

class Vector2d:
    """
    A two-dimensional vector class

        >>> v1 = Vector2d(3, 4)
        >>> print(v1.x, v1.y)
        3.0 4.0
        >>> x, y = v1
        >>> x, y
        (3.0, 4.0)
        >>> v1
        Vector2d(3.0, 4.0)
        >>> v1_clone = eval(repr(v1))
        >>> v1 == v1_clone
        True
        >>> print(v1)
        (3.0, 4.0)
            >>> octets = bytes(v1)
        >>> octets
        b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
        >>> abs(v1)
        5.0
        >>> bool(v1), bool(Vector2d(0, 0))
        (True, False)


    Test of ``.frombytes()`` class method:

        >>> v1_clone = Vector2d.frombytes(bytes(v1))
        >>> v1_clone
        Vector2d(3.0, 4.0)
        >>> v1 == v1_clone
        True


    Tests of ``format()`` with Cartesian coordinates:

        >>> format(v1)
        '(3.0, 4.0)'
        >>> format(v1, '.2f')
        '(3.00, 4.00)'
        >>> format(v1, '.3e')
        '(3.000e+00, 4.000e+00)'


    Tests of the ``angle`` method::

        >>> Vector2d(0, 0).angle()
        0.0
        >>> Vector2d(1, 0).angle()
        0.0
        >>> epsilon = 10**-8
        >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
        True
        >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
        True


    Tests of ``format()`` with polar coordinates:

        >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
        '<1.414213..., 0.785398...>'
        >>> format(Vector2d(1, 1), '.3ep')
        '<1.414e+00, 7.854e-01>'
        >>> format(Vector2d(1, 1), '0.5fp')
        '<1.41421, 0.78540>'


    Tests of `x` and `y` read-only properties:
        >>> v1.x, v1.y
        (3.0, 4.0)
        >>> v1.x = 123
        Traceback (most recent call last):
        ...
        AttributeError: can't set attribute 'x'


    Tests of hashing:

        >>> v1 = Vector2d(3, 4)
        >>> v2 = Vector2d(3.1, 4.2)
        >>> len({v1, v2})
        2
    """
    __match_args__ = ('x', 'y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

### Private and “Protected” Attributes in Python
- there only exists a simple mechanism to prevent accidental overwriting of a “private” attribute in a subclass
- if we name a private attribute with two underscores in the beginning like `__mood`, then it's name gets mangled with the class name like `_Dog__mood` in the dict of the class
    - this is to prevent accidental over-writing (name clash) this attribute in a subclass
    - if we had the same attribute in a subclass, it would become `_Beagle__mood`
    - name mangling is about safety, not security

In [11]:
# showing this for Vector2d where we have self.__x and self.__y:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [12]:
# we can read the private attribute directly (this is about safety, not security)
v1._Vector2d__x

3.0

In [14]:
# if we know about the private attribute __x, we can also set it even though it is protected as a property
v1._Vector2d__x = 7
v1._Vector2d__x

7

Using single underscores don't cause any name mangling, but are used as well to indicate privacy. There's a convention of not calling such attributes from outside of the class.

### Saving Memory with `__slots__`
- every object has a `__dict__` attribute that stores it's attributes
- but as every dict it has a significant memory overhead
- but it we define a class attribute named `__slots__` holding a sequence of attribute names, Python uses an alternative storage model for the instance attributes which uses less memory than a dict
- make sense only when handling a very large number of instances

In [15]:
class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()
p.__dict__ # those objects don't have a __dict__ attribute

AttributeError: 'Pixel' object has no attribute '__dict__'

In [16]:
p.x = 10
p.y = 20
p.color = 'red' # trying to set an attribute not listed in __slots__ raises AttributeError

AttributeError: 'Pixel' object has no attribute 'color'

In [18]:
class OpenPixel(Pixel):
    pass

op = OpenPixel()

op.__dict__ # instances of a subclass sof Pixel do have a dict (empty because the subclass doesn't declare any arguments of its own)

{}

In [19]:
# setting an attribute that is named in the parent classes' __slots__ doesn't appear in the object's __dict__
op.x = 8
op.__dict__

{}

In [ ]:
op.x # but it is stored in the hidden array of references in the instance

In [ ]:
op.color = 'green' # setting an attribute not named in the parent classes' __slots__, is saved in the instance's __dict__

To make sure that instances of a subclass have no `__dict__`, we must declare `__slots__` again in the subclass.

Another attribute to know is `__weakref__` if an object should support weak references.

### Overriding Class Attributes
- class attributes can be used as default values for instance attributes


In [ ]:
class Vector2d:
    typecode = 'd' # class attribute

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +       # the typecode is accessed with self.typecode which converts it into an instance attribute
                bytes(array(self.typecode, self)))

In [20]:
# we could change the instance attribute per subclass
v1 = Vector2d(1.1, 2.2)
v1.typecode = 'f'